In [1]:
import numpy as np
import integral_image_transform
import sklearn.datasets

def get_faces():
    return sklearn.datasets.fetch_olivetti_faces()

images = get_faces()['images']
labels = get_faces()['target']

In [2]:
def integral_transform(img, square = True, ravelled = False, dims = None):
    if square and ravelled:
        side_len = int(len(img) ** .5)
        img_reshaped = np.reshape(img, (side_len, side_len))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img_reshaped[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img_reshaped[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img_reshaped[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img_reshaped[j][i-j]
    if square and not ravelled:
        side_len = int(len(img))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img[j][i-j]

    return ans




In [89]:
image_test = integral_transform(images[0])

In [76]:
import math

def compute_two_rectangle_features_slow(integral_image):
    # Calculating at every point and scale for now
    # TODO can exchange height and width for some 24x24 window thing
    height,width = integral_image.shape # number of rows, number of columns
#     features = {}
    features = np.zeros((height,width,height,width))
    
    # Vertical (side by side) rectangles
    for i in range(height): # Starting x
        for j in range(width-2): # Starting y
            for k in range(1,height-i): # Height of rectangles
                for z in range(1,int(math.floor((width-j)/2.0))): # Width of rectangles
                    left = integral_image[i+k,j+z]- integral_image[i,j]
                    right = integral_image[i+k,j+2*z]- integral_image[i,j+z]
#                     features[(i,j,k,z)] = left-right
                    features[i,j,k,z] = left-right
    
    return features


features = compute_two_rectangle_features_slow(image_test)
# Takes 2.4 seconds this slow way
# 1.95 seconds when numpy array instead of dictionary

1 loop, best of 3: 1.95 s per loop


In [88]:
def make_array(integral_image):
    # Helper function for the vertical rectangle iterator
    # Makes a vector of the necessary proportions so np.ndenumerate can be used
    # Calculating at every point and scale for now
    height,width = integral_image.shape # number of rows, number of columns
    output = []
    first,second,third,fourth
    # Vertical (side by side) rectangles
    # Note: numpy.arange is much worse
    for i in range(height): # Starting x
        output.append([])
        for j in range(width-2): # Starting y
            output[i].append([])
            for k in range(height-i-1): # Height of rectangles
                output[i][j].append([])
                for z in range(int(math.floor((width-j)/2.0))-1): # Width of rectangles
                    output[i][j][k].append(0)
                    
    return np.array(output)

helper_array = make_array(image_test)

print helper_array.shape

(64, 62)


In [87]:
def compute_two_rectangle_features_vectorized(integral_image,template_array):
    # Calculating at every point and scale for now
    height,width = integral_image.shape # number of rows, number of columns
#     features = {}
    features = np.zeros((height,width,height,width))
    
    # Vertical (side by side) rectangles
    for index, x in np.ndenumerate(template_array):
        print 'index',index
        i,j,k,z = index[0],index[1],index[2],index[3]
        try:
            left = integral_image[i+k,j+z]- integral_image[i,j]
            right = integral_image[i+k,j+2*z]- integral_image[i,j+z]
            features[i,j,k,z] = left-right
        except IndexError:
            pass
    
    return features

%timeit new_features = compute_two_rectangle_features_vectorized(\
                                            image_test,helper_array)

index (0, 0)


IndexError: tuple index out of range

In [67]:
print features[(0,0,20,20)]
print new_features[0,0,20,20]
# print features[0,0,0,0]
# print new_features[0,0,0,0]

0.388429731131
0.388429731131


In [53]:
test = np.zeros((4,4,4,4))

for index, x in np.ndenumerate(test):
    print index[0],index[1],index[2],index[3]

0 0 0 0
0 0 0 1
0 0 0 2
0 0 0 3
0 0 1 0
0 0 1 1
0 0 1 2
0 0 1 3
0 0 2 0
0 0 2 1
0 0 2 2
0 0 2 3
0 0 3 0
0 0 3 1
0 0 3 2
0 0 3 3
0 1 0 0
0 1 0 1
0 1 0 2
0 1 0 3
0 1 1 0
0 1 1 1
0 1 1 2
0 1 1 3
0 1 2 0
0 1 2 1
0 1 2 2
0 1 2 3
0 1 3 0
0 1 3 1
0 1 3 2
0 1 3 3
0 2 0 0
0 2 0 1
0 2 0 2
0 2 0 3
0 2 1 0
0 2 1 1
0 2 1 2
0 2 1 3
0 2 2 0
0 2 2 1
0 2 2 2
0 2 2 3
0 2 3 0
0 2 3 1
0 2 3 2
0 2 3 3
0 3 0 0
0 3 0 1
0 3 0 2
0 3 0 3
0 3 1 0
0 3 1 1
0 3 1 2
0 3 1 3
0 3 2 0
0 3 2 1
0 3 2 2
0 3 2 3
0 3 3 0
0 3 3 1
0 3 3 2
0 3 3 3
1 0 0 0
1 0 0 1
1 0 0 2
1 0 0 3
1 0 1 0
1 0 1 1
1 0 1 2
1 0 1 3
1 0 2 0
1 0 2 1
1 0 2 2
1 0 2 3
1 0 3 0
1 0 3 1
1 0 3 2
1 0 3 3
1 1 0 0
1 1 0 1
1 1 0 2
1 1 0 3
1 1 1 0
1 1 1 1
1 1 1 2
1 1 1 3
1 1 2 0
1 1 2 1
1 1 2 2
1 1 2 3
1 1 3 0
1 1 3 1
1 1 3 2
1 1 3 3
1 2 0 0
1 2 0 1
1 2 0 2
1 2 0 3
1 2 1 0
1 2 1 1
1 2 1 2
1 2 1 3
1 2 2 0
1 2 2 1
1 2 2 2
1 2 2 3
1 2 3 0
1 2 3 1
1 2 3 2
1 2 3 3
1 3 0 0
1 3 0 1
1 3 0 2
1 3 0 3
1 3 1 0
1 3 1 1
1 3 1 2
1 3 1 3
1 3 2 0
1 3 2 1
1 3 2 2
1 3 2 3
1 3 3 0


In [51]:
a = [1,2,3]

def test_fnc(i,j,test_list):
#     print 'test_list',test_list
    return test_list[i]+test_list[j]

vfunc = np.vectorize(test_fnc)
print vfunc(1,2)

# Old way, doesn't work here
# np.fromfunction(test_fnc, (3,3),dtype=int,test_list=a)

test_list 1


IndexError: invalid index to scalar variable.

In [ ]:
################ Does not work

def compute_two_rectangle_features_vectorized(integral_image):
    # Calculating at every point and scale for now
    # TODO can exchange height and width for some 24x24 window thing
    height,width = integral_image.shape # number of rows, number of columns
    features = np.zeros((height,width,height,width))
    
    # Vertical (side by side) rectangles
    def make_vertical_rect(i,j,k,z):
#         print 'i',i,'j',j,'k',k,'z',z
#         print 'j',j
        try:
            left = integral_image[i+k,j+z]- integral_image[i,j]
            right = integral_image[i+k,j+2*z]- integral_image[i,j+z]
            return left-right
        except IndexError:
            return 0.0
    
    return np.fromfunction(make_vertical_rect,(height-1,width-1,height-1,\
            width),dtype=int)

def compute_two_rectangle_features_vectorized(integral_image):
    # Calculating at every point and scale for now
    height,width = integral_image.shape # number of rows, number of columns
#     features = {}
    features = np.zeros((height,width,height,width))
    
    # Vertical (side by side) rectangles
    for index, x in np.ndenumerate(features):
        i,j,k,z = index[0],index[1],index[2],index[3]
        try:
            left = integral_image[i+k,j+z]- integral_image[i,j]
            right = integral_image[i+k,j+2*z]- integral_image[i,j+z]
            features[i,j,k,z] = left-right
        except:
            pass
    
    return features

                    